In [1]:
###日志 logger##20220825A######
import logging
import sys
import time

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')

def make_logger(name, log_level=logging.DEBUG, log_file="log.txt", file_mode="a"): #w写 a追加
    logger = logging.getLogger(name)
    logger.setLevel(level=log_level)
    handler = logging.FileHandler(log_file, mode=file_mode)
    handler.setLevel(level=log_level)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(level=log_level)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

name = 'restAcc'
curDate = time.strftime("%Y%m%d")
logger = make_logger(name, log_level=logging.DEBUG, log_file= "../logs/%s_%s.log" % (name, curDate) )
# logging.basicConfig(level=your_level)  logger.debug info  warning  error critical 
# logging.disable(sys.maxint) # 完全禁用日志记录

version = "1.0.1"
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****日志 logger***** ')

Now: 1.0.1 : 2022-10-25 02:39:22 *****日志 logger***** 


In [4]:
###时间戳 日期 datetime###
from datetime import datetime, timedelta, timezone 
import time

print(1, f'{datetime.fromtimestamp(1662076800121//1000) }') #秒数 转日期 转字串
print(2, f'{datetime.fromtimestamp(1662076800) }' ) #datetime.date.min .max .resolution
print(3, f'{datetime(2022,1,1,0,0,0) } , { datetime.strptime("2020-9-19 18:19:59", "%Y-%m-%d %H:%M:%S") }')

print(4, datetime.now() , datetime.now().timestamp() , datetime.now().strftime('%a, %b %d %Z %w,%W,%j %H:%M:%S %p , %x %X') ) #d.timestamp(),d.today(), d.year,d.timetuple()
print(5, datetime.now() - timedelta(days=1,hours=2,minutes=3,seconds=4,microseconds=5) )
print(6, (datetime.now() - datetime(2022,1,1)).days ) # .days  .seconds .microseconds  

print(7,  f'{time.strftime("%Y-%m-%d %H:%M:%S")} , {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1657088760999//1000)) = }' ) #2022-07-03 10:55:25
print(8, f'{int(time.mktime(time.strptime("2022-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))-1 = }000') #timeStamp 1656816925  2022-07-15:1657814400
print(9, f'{time.time() = } , {int(time.time()) = } , { int(time.time()) + 5}999' ) 


# rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
# rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")


1 2022-09-02 00:00:00
2 2022-09-02 00:00:00
3 2022-01-01 00:00:00 , 2020-09-19 18:19:59
4 2022-09-27 17:05:01.722789 1664298301.722791 Tue, Sep 27  2,39,270 17:05:01 PM , 09/27/22 17:05:01
5 2022-09-26 15:01:57.722966
6 269
7 2022-09-27 17:05:01 , time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1657088760999//1000)) = '2022-07-06 06:26:00'
8 int(time.mktime(time.strptime("2022-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))-1 = 1640995199000
9 time.time() = 1664298301.7234643 , int(time.time()) = 1664298301 , 1664298306999


In [5]:
###账户 import###
import hashlib
import hmac
import time
import requests
import json
from datetime import datetime 
import pandas as pd
import sqlite3
import math
import sys
import threading
import logging
import os
import pickle 
import jwt

# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)		# 显示行数
pd.set_option('display.max_columns', 500)	# 显示列数
pd.set_option('display.width', 1000)		# 显示宽度

# worthDir = '/home/at/ipynb/pd'
# dbfile = '%s/%s' % ('/mnt/c/milu/docx/strategy/v2.1/binance', 'binance.db')

# worthDir = '/home/at/py/ipynb/'
# dbfile = '%s/%s' % ('/mnt/e/milu/py', 'binance.db')

worthDir = r'../pkl'
dbfile = r'%s/%s' % (worthDir, 'binance.db')

serverUrl = r'http://testnet.binancefuture.com'
serverTimeadj = 10
newClientOrderId = ''
orderId = ''

url = f'{serverUrl}/fapi/v1/time'
timestamp = int(round((time.time() ) * 1000) )
res = requests.get(url)
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else :
    serverTimeadj = ((rt['serverTime']- timestamp) ) //1000

df_balance = pd.DataFrame()
df_account = pd.DataFrame()
df_position = pd.DataFrame()
df_order = pd.DataFrame()
df_trade = pd.DataFrame()

version = "1.0.1"
print(sys.version)
# print(sys.executable)
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} +{serverTimeadj}*****账户 import***** ')

3.10.4 (main, Aug 22 2022, 22:50:25) [GCC 9.4.0]
Now: 1.0.1 : 2022-09-27 17:05:02 +0*****账户 import***** 


In [6]:
###查询账户余额 balance###QryBalance###
def QryBalance():

    global df_balance

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v2/balance'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)
    # print(f'{rt}')
    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        rt_df = pd.DataFrame([rt[1] ])
        # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000)) 
        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00")
        rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

        rt_df['availableBalance']  = pd.to_numeric(rt_df['availableBalance'] , errors='coerce', downcast='float')
        rt_df['balance']  = pd.to_numeric(rt_df['balance'] , errors='coerce', downcast='float')
        rt_df['crossWalletBalance']  = pd.to_numeric(rt_df['crossWalletBalance'] , errors='coerce', downcast='float')
        rt_df['crossUnPnl']  = pd.to_numeric(rt_df['crossUnPnl'] , errors='coerce', downcast='float')
        rt_df['maxWithdrawAmount']  = pd.to_numeric(rt_df['maxWithdrawAmount'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        df_balance = pd.concat([df_balance, rt_df ], ignore_index=True)
        df_b = df_balance[['asset','balance','availableBalance','updateTime','queryTime', ] ]

        conn = sqlite3.connect(dbfile)
        # df_balance = pd.read_sql(' select * from balance ', conn) 
        rt_df.to_sql('balance', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_balance.to_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )
        # df_bal = pd.read_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )

        logger.info(f'df_b :\n{df_b}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****查询账户余额 balance###QryBalance***** ')

Now: 1.0.1 : 2022-09-27 17:05:02 *****查询账户余额 balance###QryBalance***** 


In [7]:
###查询账户数据v2 account###余额assets+持仓positions###QryAccount###
def QryAccount():

    global df_account
    global df_position

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v2/account'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)

    # logger.info(f'res.text :\n{res.text}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :

        for ass in rt["assets"]:

            if ass['asset'] != 'USDT':
                continue 

            rt_df = pd.DataFrame([ass])

            # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

            rt_df['availableBalance']  = pd.to_numeric(rt_df['availableBalance'] , errors='coerce', downcast='float')
            rt_df['crossUnPnl']  = pd.to_numeric(rt_df['crossUnPnl'] , errors='coerce', downcast='float')
            rt_df['crossWalletBalance']  = pd.to_numeric(rt_df['crossWalletBalance'] , errors='coerce', downcast='float')
            rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
            rt_df['marginBalance']  = pd.to_numeric(rt_df['marginBalance'] , errors='coerce', downcast='float')
            rt_df['maxWithdrawAmount']  = pd.to_numeric(rt_df['maxWithdrawAmount'] , errors='coerce', downcast='float')
            rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
            rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
            rt_df['unrealizedProfit']  = pd.to_numeric(rt_df['unrealizedProfit'] , errors='coerce', downcast='float')
            rt_df['walletBalance']  = pd.to_numeric(rt_df['walletBalance'] , errors='coerce', downcast='float')

            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            df_account = pd.concat([df_account,rt_df ], ignore_index=True)
            df_a = df_account[['asset','walletBalance','availableBalance','maintMargin','marginBalance','unrealizedProfit','updateDate','queryTime', ] ]

            conn = sqlite3.connect(dbfile)
            # df_account = pd.read_sql(' select * from account ', conn) 
            rt_df.to_sql('account', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_account.to_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
            # df_acc = pd.read_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )

            logger.info(f'df_a :\n{df_a}')

        for posi in rt["positions"]:

            if not abs(float(posi['positionAmt'])) > 0 :
                continue 

            rt_df = pd.DataFrame([posi])

            # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

            rt_df['initialMargin']  = pd.to_numeric(rt_df['initialMargin'] , errors='coerce', downcast='float')
            rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
            rt_df['unrealizedProfit']  = pd.to_numeric(rt_df['unrealizedProfit'] , errors='coerce', downcast='float')
            rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
            rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
            rt_df['entryPrice']  = pd.to_numeric(rt_df['entryPrice'] , errors='coerce', downcast='float')
            rt_df['maxNotional']  = pd.to_numeric(rt_df['maxNotional'] , errors='coerce', downcast='float')
            rt_df['bidNotional']  = pd.to_numeric(rt_df['bidNotional'] , errors='coerce', downcast='float')
            rt_df['askNotional']  = pd.to_numeric(rt_df['askNotional'] , errors='coerce', downcast='float')
            rt_df['positionAmt']  = pd.to_numeric(rt_df['positionAmt'] , errors='coerce', downcast='float')

            rt_df['leverage']  = pd.to_numeric(rt_df['leverage'] , errors='coerce', downcast='integer')

            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            df_position = pd.concat([df_position, rt_df ], ignore_index=True)
            df_p = df_position[['symbol','initialMargin','maintMargin','unrealizedProfit','positionInitialMargin','openOrderInitialMargin','leverage','entryPrice','maxNotional','positionAmt','notional','updateDate','queryTime', ] ]

            conn = sqlite3.connect(dbfile)
            # df_position = pd.read_sql(' select * from position ', conn) 
            rt_df.to_sql('position', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_position.to_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )
            # df_posi = pd.read_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )

            logger.info(f'df_p :\n{df_p}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****查询账户数据 account###余额assets+持仓positions###QryAccount***** ')        


Now: 1.0.1 : 2022-09-27 17:05:02 *****查询账户数据 account###余额assets+持仓positions###QryAccount***** 


In [8]:
###交易 order###订单###TradeFuture###
def TradeFuture(symbol='btcusdt', side='BUY', price=19000.1, quantity=0.001, orderType='LIMIT', timeInForce='GTC'):
    # , callbackRate=5, stopPrice=20000, activationPrice=20000, reduceOnly='false', priceProtect='false', closePosition='true'

    global df_order
    global df_trade
    global newClientOrderId
    global orderId

    timestamp = int(round((time.time() +serverTimeadj+1) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/order'
    urlTest = "http://httpbin.org/post"
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = 'btcusdt'  #交易对 
    # side = 'BUY'        #买卖方向 BUY 买入, SELL 卖出
    # orderType = 'LIMIT'      #订单类型 LIMIT限价单, MARKET市价单, STOP止损限价单, STOP_MARKET止损市价单, TAKE_PROFIT止盈限价单, TAKE_PROFIT_MARKET止盈市价单, TRAILING_STOP_MARKET跟踪止损单,
    # price = 19000.1  #委托价格
    # quantity = 0.001  #下单数量
    # timeInForce = 'GTC' #有效方法 GTC-Good Till Cancel 成交为止, IOC-Immediate or Cancel 无法立即成交(吃单)的部分就撤销, FOK-Fill or Kill 无法全部立即成交就撤销, GTX-Good Till Crossing 无法成为挂单方就撤销

    reduceOnly = 'false' #仅减仓 false true 
    workingType = 'CONTRACT_PRICE'  #stopPrice 触发类型: MARK_PRICE(标记价格), CONTRACT_PRICE(合约最新价). 默认 CONTRACT_PRICE
    priceProtect = 'false' #条件单触发保护："TRUE","FALSE", 默认"FALSE". 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    newOrderRespType = "ACK"  #"ACK", "RESULT", 默认 "ACK"

    sideDict = {'BUY':'B', 'SELL':'S', }
    typeDict = {'LIMIT':'LMT', 'MARKET':'MKT', 'STOP':'STP','STOP_MARKET':'STP','TAKE_PROFIT':'TKP','TAKE_PROFIT_MARKET':'TPM','TRAILING_STOP_MARKET':'TSM', }
    reduceOnlyDict = {'false':0, 'true':1,}
    workingTypeDict = {'CONTRACT_PRICE':'C', 'MARK_PRICE':'M',}
    priceProtectDict = {'false':0, 'true':1,}
    newOrderRespTypeDict = {'ACK':'A', 'RESULT':'R',}

    positionSide = 'BOTH' #持仓方向，单向持仓模式下非必填，默认且仅可填BOTH;在双向持仓模式下必填,且仅可选择 LONG 或 SHORT, BOTH 单一持仓方向, LONG 多头(双向持仓下), SHORT 空头(双向持仓下)
    callbackRate = 5 # 追踪止损回调比例，可取值范围[0.1, 5],其中 1代表1% ,仅TRAILING_STOP_MARKET 需要此参数
    stopPrice = 20000   #触发价, 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    closePosition = 'true' #触发后全部平仓，true, false；仅支持STOP_MARKET和TAKE_PROFIT_MARKET；不与quantity合用；自带只平仓效果，不与reduceOnly 合用
    activationPrice = 20000 #追踪止损激活价格，仅TRAILING_STOP_MARKET 需要此参数, 默认为下单当前市场价格(支持不同workingType)

    closePositionDict = {'false':0, 'true':1,}
    positionSideDict = {'BOTH':'B', 'LONG':'L', 'SHORT':'S', }

    tailStr = '%d%s%d%s%d%s' % (reduceOnlyDict[reduceOnly], workingTypeDict[workingType], priceProtectDict[priceProtect], newOrderRespTypeDict[newOrderRespType], closePositionDict[closePosition], positionSideDict[positionSide] )
    symbolStr = '%s-------' % (symbol.lower() )
    tacticName = 'T0'
    newClientOrderId = '%s%s%s%s%s%s%s' % (tacticName, sideDict[side], time.strftime("%Y%m%d%H%M%S"), typeDict[orderType], symbolStr[:7], timeInForce, tailStr )  #自定义订单号 ^[\.A-Z\:/a-z0-9_-]{1,36}$    2+1+14+3 +7+3+6

    # orderType = 'LIMIT'      #订单类型 LIMIT 限价单, MARKET 市价单, STOP 止损限价单, STOP_MARKET 止损市价单, TAKE_PROFIT 止盈限价单, TAKE_PROFIT_MARKET 止盈市价单, TRAILING_STOP_MARKET 跟踪止损单,

    if orderType == 'LIMIT':
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))

    elif orderType == 'MARKET':
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&quantity=%f&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,quantity,recvWindow,str(timestamp))

    else:
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))



    secret = secret_key
    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    # params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    params = {"newClientOrderId":newClientOrderId,"symbol":symbol,"side":side,"type":orderType,"price":price,"quantity":quantity,"timeInForce":timeInForce, "recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}

    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)

    # res = requests.post(url, headers=headers, params=params) #, params=params  urlTest  url2 
    res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    logger.info(f'url2 : {url2}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        del rt['cumQty'] #弃用
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values:
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        orderId = rt['orderId']

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from orders ', conn) 
        rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')
        
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****交易 order###订单###TradeFuture***** ')        


Now: 1.0.1 : 2022-09-27 17:05:03 *****交易 order###订单###TradeFuture***** 


In [9]:
###查询订单 orders###订单合并###QryOrder###
def QryOrder(orderId, symbol='btcusdt'):

    global df_order
    global newClientOrderId


    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/order'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
    # origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    # orderId = 3204222910
    # origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    query_string = 'symbol=%s&orderId=%s&recvWindow=%s&timestamp=%s' % (symbol,orderId,recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"symbol":symbol,"orderId":orderId,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)
        
        if  not df_order.empty and orderId in df_order['orderId'].values:  #and newClientOrderId in df_order['clientOrderId'].values 
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == orderId ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_balance = pd.read_sql(' select * from orders ', conn) 
        rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****查询订单 orders###订单合并###QryOrder***** ')  

Now: 1.0.1 : 2022-09-27 17:05:03 *****查询订单 orders###订单合并###QryOrder***** 


In [10]:
###撤销订单 order###订单合并###CancelOrder###
def CancelOrder(orderId, symbol='btcusdt'):

    global df_order
    
    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/order'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
    # origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    # orderId = 3204222910
    # origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    query_string = 'symbol=%s&orderId=%s&recvWindow=%s&timestamp=%s' % (symbol,orderId,recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"symbol":symbol,"orderId":orderId,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.delete(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        del rt['cumQty'] #弃用
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values: 
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from orders ', conn) 
        rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****撤销订单 order###订单合并###CancelOrder***** ')  

Now: 1.0.1 : 2022-09-27 17:05:03 *****撤销订单 order###订单合并###CancelOrder***** 


In [11]:
###撤销全部订单 allOpenOrders###订单合并###CancelAllOpenOrders###
def CancelAllOpenOrders(symbol='btcusdt'):

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/allOpenOrders'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
    # origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    # orderId = 3204222910
    # origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.delete(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        if rt['code'] == '200':
            logger.info(f'res : {res.text}')
        else :
            logger.error(f'res : {res.text}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****撤销全部订单 allOpenOrders###订单合并###CancelAllOpenOrders***** ')  

Now: 1.0.1 : 2022-09-27 17:05:03 *****撤销全部订单 allOpenOrders###订单合并###CancelAllOpenOrders***** 


In [12]:
###查询当前全部挂单 openOrders###订单合并###QryOpenOrders###
def QryOpenOrders():

    global df_order
    global orderId
    global newClientOrderId

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/openOrders'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
    # origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    # orderId = 3204222910
    # origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        for order in rt:
            rt_df = pd.DataFrame([order])

            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
            # rt_df['time'] = timestamp

            rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
            rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
            rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
            rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
            rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
            rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
            # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
            # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            
            if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and orderId in df_order['orderId'].values: 
                df_order.sort_index(axis=1, ascending=True, inplace=True)
                df_order.loc[ df_order['orderId'] == orderId ] = rt_df.to_numpy()
            else:
                df_order = pd.concat([df_order,rt_df ], ignore_index=True)

            df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

            conn = sqlite3.connect(dbfile)
            # ##df_order = pd.read_sql(' select * from order ', conn) 
            rt_df.to_sql('order', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
            # ##df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_order :\n{df_order}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****查询当前全部挂单 openOrders###订单合并###QryOpenOrders***** ')  


Now: 1.0.1 : 2022-09-27 17:05:04 *****查询当前全部挂单 openOrders###订单合并###QryOpenOrders***** 


In [13]:
###用户持仓风险V2 positionRisk###get###QryPositionRisk###
def QryPositionRisk(symbol='btcusdt'):

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/positionRisk'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = "btcusdt"

    query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)

    res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else:
        logger.info(f'res : {res.text}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****用户持仓风险V2 positionRisk###get###QryPositionRisk***** ')  

Now: 1.0.1 : 2022-09-27 17:05:04 *****用户持仓风险V2 positionRisk###get###QryPositionRisk***** 


In [14]:
###最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex###
def QryPremiumIndex(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/premiumIndex'

    params = {"symbol":symbol, }

    res = requests.get(url, params=params) #
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')
    else :
        logger.info(f'res : {res.text}')  
        # {"symbol":"BTCUSDT","markPrice":"20133.32973555","indexPrice":"20133.75155317","estimatedSettlePrice":"20056.77850121","lastFundingRate":"-0.00194053","interestRate":"0.00010000","nextFundingTime":1661961600000,"time":1661961098000}

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex***** ') 

Now: 1.0.1 : 2022-09-27 17:05:04 *****最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex***** 


In [15]:
###深度信息 depth###get###QryDepth###
def QryDepth(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/depth'

    params = {"symbol":symbol, "limit":5, } 

    res = requests.get(url, params=params)  
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')
    else :
        logger.info(f'res : {res.text}')  
        # {"lastUpdateId":27532317617,"E":1664272323243,"T":1664272323241,"bids":[["20173.30","4.937"],["20173.00","0.001"],["20172.90","4.228"],["20172.50","18.436"],["20171.30","14.832"]],"asks":[["20183.40","1.430"],["20183.80","61.862"],["20184.20","128.430"],["20184.60","71.158"],["20185.00","3.459"]]}

        print(f"rt : {rt}")
        print(f"rt['bids'][0][0] : {rt['bids'][0][0]}")
        print(f"rt['bids'][0][0] : {rt['bids'][0][0]} , rt['asks'][0][0] : {rt['asks'][0][0]}")

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****###深度信息 depth###get###QryDepth###***** ') 

Now: 1.0.1 : 2022-09-27 17:05:05 *****###深度信息 depth###get###QryDepth###***** 


In [16]:
QryDepth()

2022-09-27 17:05:05,446:INFO:restAcc:957371473.py:14:QryDepth:1236: res : {"lastUpdateId":27538283623,"E":1664298305081,"T":1664298305076,"bids":[["18955.10","1.404"],["18953.20","98.954"],["18943.90","0.500"],["18940.00","0.026"],["18934.90","1.447"]],"asks":[["18955.20","0.052"],["18963.80","27.670"],["18965.70","13.393"],["18968.70","24.100"],["18970.60","99.297"]]}
2022-09-27 17:05:05,446:INFO:restAcc:957371473.py:14:QryDepth:1236: res : {"lastUpdateId":27538283623,"E":1664298305081,"T":1664298305076,"bids":[["18955.10","1.404"],["18953.20","98.954"],["18943.90","0.500"],["18940.00","0.026"],["18934.90","1.447"]],"asks":[["18955.20","0.052"],["18963.80","27.670"],["18965.70","13.393"],["18968.70","24.100"],["18970.60","99.297"]]}
rt : {'lastUpdateId': 27538283623, 'E': 1664298305081, 'T': 1664298305076, 'bids': [['18955.10', '1.404'], ['18953.20', '98.954'], ['18943.90', '0.500'], ['18940.00', '0.026'], ['18934.90', '1.447']], 'asks': [['18955.20', '0.052'], ['18963.80', '27.670'],

In [17]:
###最新价格 ticker/price###get###QryTcikerPrice###
def QryTcikerPrice(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/ticker/price'

    params = {"symbol":symbol, }

    res = requests.get(url, params=params) #
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')  
    else :
        logger.info(f'res : {res.text}')  #{"symbol":"BTCUSDT","price":"20137.10","time":1661961001841}

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****最新价格 ticker/price###get###QryTcikerPrice***** ') 

Now: 1.0.1 : 2022-09-27 17:05:05 *****最新价格 ticker/price###get###QryTcikerPrice***** 


In [18]:
###获取交易规则和交易对 U本位 rateLimits filters###get###QryExchangeInfo###
def QryExchangeInfo():
    url = f'{serverUrl}/fapi/v1/exchangeInfo'
    res = requests.get(url) #
    rt = json.loads(res.text)

    with open(f"{worthDir}/exchangeInfo_u.pkl","wb") as handle:
        pickle.dump(rt, handle)   #保存

    # with open(f"{worthDir}/exchangeInfo_u.pkl","rb") as handle:
    #     exchangeInfo_u = pickle.load(handle)  #读取

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')  
    else :
        logger.info(f'res : {res.text}')  #

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****获取交易规则和交易对 U本位 rateLimits filters###get###QryExchangeInfo***** ') 

Now: 1.0.1 : 2022-09-27 17:05:05 *****获取交易规则和交易对 U本位 rateLimits filters###get###QryExchangeInfo***** 


In [19]:
eufile = f"{worthDir}/exchangeInfo_u.pkl"
if os.path.exists(eufile):
    with open(eufile,"rb") as handle:
        exchangeInfo_u = pickle.load(handle)  #读取
    
# print(f"{exchangeInfo_u }")  # rateLimits symbols   
# print(f"{exchangeInfo_u['symbols'] }")   

for item in exchangeInfo_u['symbols']:
    if item['symbol'] == 'BTCUSDT' :
        for ite in item['filters']:
            if ite['filterType'] == 'LOT_SIZE' :             
                print( len(ite['minQty'].split(".")[1])  )
                


3


In [21]:
###交易清仓 order###订单###Clearance###
def Clearance(symbol='btcusdt', side='BUY', orderType='TAKE_PROFIT_MARKET', stopPrice=20300):

    global df_order

    timestamp = int(round((time.time() +serverTimeadj+1) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/order'
    urlTest = "http://httpbin.org/post"
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = 'btcusdt'  #交易对 
    # side = 'BUY'        #买卖方向 BUY 买入, SELL 卖出
    # orderType = 'TAKE_PROFIT_MARKET'      #订单类型 LIMIT限价单, MARKET市价单, STOP止损限价单, STOP_MARKET止损市价单, TAKE_PROFIT止盈限价单, TAKE_PROFIT_MARKET止盈市价单, TRAILING_STOP_MARKET跟踪止损单,
    price = 19000.1  #委托价格
    quantity = 0.001  #下单数量
    timeInForce = 'GTC' #有效方法 GTC-Good Till Cancel 成交为止, IOC-Immediate or Cancel 无法立即成交(吃单)的部分就撤销, FOK-Fill or Kill 无法全部立即成交就撤销, GTX-Good Till Crossing 无法成为挂单方就撤销

    reduceOnly = 'false' #仅减仓 false true 
    workingType = 'CONTRACT_PRICE'  #stopPrice 触发类型: MARK_PRICE(标记价格), CONTRACT_PRICE(合约最新价). 默认 CONTRACT_PRICE
    priceProtect = 'true' #条件单触发保护："TRUE","FALSE", 默认"FALSE". 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    newOrderRespType = "ACK"  #"ACK", "RESULT", 默认 "ACK"

    sideDict = {'BUY':'B', 'SELL':'S', }
    typeDict = {'LIMIT':'LMT', 'MARKET':'MKT', 'STOP':'STP','STOP_MARKET':'STP','TAKE_PROFIT':'TKP','TAKE_PROFIT_MARKET':'TPM','TRAILING_STOP_MARKET':'TSM', }
    reduceOnlyDict = {'false':0, 'true':1,}
    workingTypeDict = {'CONTRACT_PRICE':'C', 'MARK_PRICE':'M',}
    priceProtectDict = {'false':0, 'true':1,}
    newOrderRespTypeDict = {'ACK':'A', 'RESULT':'R',}

    positionSide = 'BOTH' #持仓方向，单向持仓模式下非必填，默认且仅可填BOTH;在双向持仓模式下必填,且仅可选择 LONG 或 SHORT, BOTH 单一持仓方向, LONG 多头(双向持仓下), SHORT 空头(双向持仓下)
    callbackRate = 5 # 追踪止损回调比例，可取值范围[0.1, 5],其中 1代表1% ,仅TRAILING_STOP_MARKET 需要此参数
    # stopPrice = 20300   #触发价, 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    closePosition = 'true' #触发后全部平仓，true, false；仅支持 STOP_MARKET 和 TAKE_PROFIT_MARKET ；不与quantity合用；自带只平仓效果，不与reduceOnly 合用
    activationPrice = 20000 #追踪止损激活价格，仅TRAILING_STOP_MARKET 需要此参数, 默认为下单当前市场价格(支持不同workingType)

    closePositionDict = {'false':0, 'true':1,}
    positionSideDict = {'BOTH':'B', 'LONG':'L', 'SHORT':'S', }

    tailStr = '%d%s%d%s%d%s' % (reduceOnlyDict[reduceOnly], workingTypeDict[workingType], priceProtectDict[priceProtect], newOrderRespTypeDict[newOrderRespType], closePositionDict[closePosition], positionSideDict[positionSide] )
    symbolStr = '%s-------' % (symbol.lower() )
    tacticName = 'T0'
    newClientOrderId = '%s%s%s%s%s%s%s' % (tacticName, sideDict[side], time.strftime("%Y%m%d%H%M%S"), typeDict[orderType], symbolStr[:7], timeInForce, tailStr )  #自定义订单号 ^[\.A-Z\:/a-z0-9_-]{1,36}$    2+1+14+3 +7+3+6

    # query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))
    query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&stopPrice=%f&priceProtect=%s&closePosition=%s&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,stopPrice,priceProtect,closePosition,timeInForce,recvWindow,str(timestamp))
    secret = secret_key
    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    # params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    # params = {"newClientOrderId":newClientOrderId,"symbol":symbol,"side":side,"type":orderType,"price":price,"quantity":quantity,"timeInForce":timeInForce, "recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    params = {"newClientOrderId":newClientOrderId,"symbol":symbol,"side":side,"type":orderType,"stopPrice":stopPrice,"priceProtect":priceProtect,"closePosition":closePosition,"timeInForce":timeInForce, "recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}

    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)

    # res = requests.post(url, headers=headers, params=params) #, params=params  urlTest  url2 
    res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    logger.info(f'url2 : {url2}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        del rt['cumQty'] #弃用
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values:
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        orderId = rt['orderId']

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from orders ', conn) 
        rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')
        
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****交易清仓 order###订单###Clearance***** ')        


Now: 1.0.1 : 2022-09-27 17:05:08 *****交易清仓 order###订单###Clearance***** 


In [22]:
###查询组合 QryBalAccOrd ###
def QryBalAccOrd(n):

    logger.info(f'{time.strftime("%Y-%m-%d %H:%M:%S")} 查询组合: {n} , 余额+账户数据/持仓+订单')
    
    ###1. 查询###余额 balance###
    QryBalance()
    print(f'\n df_balance: \n{df_balance} \n')  #

    ###2. 查询账户数据 account###余额assets+持仓positions###QryAccount###
    QryPremiumIndex()
    QryAccount()

    print(f'\n df_account: \n{df_account} \n')
    print(f'\n df_position: \n{df_position} \n')

    ###3. 查询订单 orders###订单合并###
    # QryOrder(orderId)  #3205875652  orderId
    QryOpenOrders()

    ###4. 用户持仓风险V2 positionRisk###get###QryPositionRisk###
    QryTcikerPrice()
    QryPositionRisk()



In [23]:
###查询组合 QryBalAccOrd_Thd ###多线程 并发###
def QryBalAccOrd_Thd(n):

    logger.info(f'{time.strftime("%Y-%m-%d %H:%M:%S")} 查询组合: {n} , 余额+账户数据/持仓+订单')
    
    ###1. 查询###余额 balance###
    t1 = threading.Thread(target=QryBalance, args=())
    # print(f'\n df_balance: \n{df_balance} \n')  #

    ###2. 查询账户数据 account###余额assets+持仓positions###QryAccount###
    t2 = threading.Thread(target=QryPremiumIndex, args=())
    t3 = threading.Thread(target=QryAccount, args=())
    # print(f'\n df_account: \n{df_account} \n')
    # print(f'\n df_position: \n{df_position} \n')

    ###3. 查询订单 orders###订单合并###
    # QryOrder(orderId)  #3205875652  orderId
    t4 = threading.Thread(target=QryOpenOrders, args=())

    ###4. 用户持仓风险V2 positionRisk###get###QryPositionRisk###
    t5 = threading.Thread(target=QryTcikerPrice, args=())
    t6 = threading.Thread(target=QryPositionRisk, args=())

    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t5.start()
    t6.start()

    t1.join()
    t2.join()
    t3.join()
    t4.join()
    t5.join()
    t6.join()


In [24]:
print(f'{time.strftime("%Y-%m-%d %H:%M:%S")} *****查询 多线程###threading.Thread***** \n')
# logger.disabled = False
# %time
# orderId = '3205894565'
# print(df_account)
# print(df_position)

# CancelAllOpenOrders() # 全撤
# QryTcikerPrice()  # 最新价格
# QryPremiumIndex() # 最新标记价格 资金费率
# QryPositionRisk() # 持仓风险
# QryOpenOrders() # 全部挂单
QryAccount() # 查询账户数据v2
# QryBalance() # 余额
# QryOrder(orderId) # 查询订单

# QryBalAccOrd(0)
# Clearance(symbol='btcusdt', side='SELL', orderType='TAKE_PROFIT_MARKET', stopPrice=20300) # 清仓
# QryBalAccOrd(1)

# t1 = threading.Thread(target=QryBalAccOrd, args=(0,))
# t1.start()
# t1.join()

# QryBalAccOrd_Thd(0)

# QryExchangeInfo() # 获取交易规则和交易对 U本位

# print(f'\n df_account: \n{df_account} \n')
# print(f'\n df_position: \n{df_position} \n')
# print(f'\n df_balance: \n{df_balance} \n')  #
# print(f'\n df_order: \n{df_order} \n')  #


display(df_position)



2022-09-27 17:06:53 *****查询 多线程###threading.Thread***** 

2022-09-27 17:06:53,379:INFO:restAcc:2090940272.py:68:QryAccount:1236: df_a :
  asset  walletBalance  availableBalance  maintMargin  marginBalance  unrealizedProfit              updateDate           queryTime
0  USDT    4090.143311       4090.143311          0.0    4090.143311               0.0 2022-09-28 00:43:02.422 2022-09-28 01:06:53
2022-09-27 17:06:53,379:INFO:restAcc:2090940272.py:68:QryAccount:1236: df_a :
  asset  walletBalance  availableBalance  maintMargin  marginBalance  unrealizedProfit              updateDate           queryTime
0  USDT    4090.143311       4090.143311          0.0    4090.143311               0.0 2022-09-28 00:43:02.422 2022-09-28 01:06:53


""


In [ ]:
logger.info(f'{time.strftime("%Y-%m-%d %H:%M:%S")} ###场景一: 挂单 限价 买/卖  撤单(不成交)###')

###测试场景
# df_balance = pd.read_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )
# df_account = pd.read_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
# df_position = pd.read_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )
# df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )


###1. 查询###空仓###
QryBalAccOrd(1)

###2. 交易挂单  限 BUY order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='BUY', price=19000.3, quantity=0.001, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')
print(f'df_trade: \n{df_trade} \n')

###3. 查询###挂单###
QryBalAccOrd(2)

###4. 撤销订单 order###订单合并###cancelOrder###
CancelOrder(orderId)  #

###5. 查询###撤单###
QryBalAccOrd(3)



###6. 交易挂单 限 SELL order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='SELL', price=21000.4, quantity=0.001, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')
# print(f'df_trade: \n{df_trade} \n')

###7. 查询###挂单###
QryBalAccOrd(4)

###8. 撤销订单 order###订单合并###cancelOrder###
CancelOrder(orderId)  #

###9. 查询###撤单###
QryBalAccOrd(5)


In [ ]:
logger.info(f'{time.strftime("%Y-%m-%d %H:%M:%S")} ###场景二: 双向挂单 限价 买/卖 ( 不成交)###')
# CancelAllOpenOrders()

###10. 查询###挂单###
QryBalAccOrd(6)

###11. 交易挂单 双向挂单 BUY order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='BUY', price=19000.3, quantity=0.001, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')

###12. 查询###双向挂单###
QryBalAccOrd(7)

###13. 交易挂单 限 SELL order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='SELL', price=21000.4, quantity=0.004, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')
# print(f'df_trade: \n{df_trade} \n')

###14. 查询###挂单###
QryBalAccOrd(8)


###15. 交易挂单 双向挂单 BUY order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='BUY', price=19000.3, quantity=0.002, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')

###16. 查询###双向挂单###
QryBalAccOrd(9)

###17. 全部撤单###双向挂单###
CancelAllOpenOrders()

###18. 查询###双向挂单###
QryBalAccOrd(10)


In [39]:
logger.info(f'{time.strftime("%Y-%m-%d %H:%M:%S")} ###场景三: 挂单 市价 买/卖  (成交 持仓)###')

###19. 查询###双向挂单###
QryBalAccOrd(11)

###20. 交易成交 市 SELL order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='SELL', price=21000.5, quantity=0.010, orderType='MARKET', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')

time.sleep( 60 * 3 )

###21. 查询###成交后###
QryBalAccOrd(12)


###22. 交易成交 市 BUY order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='BUY', price=20000.5, quantity=0.030, orderType='MARKET', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')

time.sleep( 60 * 3 )

###23. 查询###成交后###
QryBalAccOrd(13)


2022-08-31 16:05:19,839:INFO:restAcc:3700459278.py:1:<cell line: 1>:355: 2022-08-31 16:05:19 ###场景三: 挂单 市价 买/卖  (成交 持仓)###
2022-08-31 16:05:19,846:INFO:restAcc:1949615123.py:4:QryBalAccOrd:355: Now: 1.0.1 : 2022-08-31 16:05:19 查询组合: 11 , 余额+账户数据/持仓+订单
2022-08-31 16:05:31,933:INFO:restAcc:584110258.py:54:QryBalance:355: df_b :
   asset      balance  availableBalance     updateTime           queryTime
0   USDT  3935.019287       3935.019287  1661843145792 2022-08-31 10:17:50
1   USDT  3935.019287       3916.019043  1661843145792 2022-08-31 10:18:38
2   USDT  3935.019287       3935.019287  1661843145792 2022-08-31 10:19:26
3   USDT  3935.019287       3914.019043  1661843145792 2022-08-31 10:25:19
4   USDT  3935.019287       3935.019287  1661843145792 2022-08-31 10:26:02
5   USDT  3935.019287       3916.019043  1661843145792 2022-08-31 12:08:16
6   USDT  3935.019287       3851.017822  1661843145792 2022-08-31 12:09:01
7   USDT  3935.019287       3851.017822  1661843145792 2022-08-31 12:09:

In [24]:
logger.info(f'{time.strftime("%Y-%m-%d %H:%M:%S")} ###场景四: 双向挂单 限价 买/卖 ( 不成交)###')
# CancelAllOpenOrders()

###24. 查询###挂单###
QryBalAccOrd(14)

###25. 交易挂单 双向挂单 BUY order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='BUY', price=19000.3, quantity=0.001, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')

###26. 查询###双向挂单###
QryBalAccOrd(15)

###27. 交易挂单 限 SELL order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='SELL', price=21000.4, quantity=0.004, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')
# print(f'df_trade: \n{df_trade} \n')

###28. 查询###挂单###
QryBalAccOrd(16)


###29. 交易挂单 双向挂单 BUY order###订单###TradeFuture
TradeFuture(symbol='btcusdt', side='BUY', price=19000.3, quantity=0.002, orderType='LIMIT', timeInForce='GTC')

print(f'df_order: \n{df_order} \n')

###30. 查询###双向挂单###
QryBalAccOrd(17)

###31. 全部撤单###双向挂单###
# CancelAllOpenOrders()

###32. 查询###双向挂单###
# QryBalAccOrd(18)


2022-09-01 16:02:58,364:INFO:restAcc:1932153918.py:1:<cell line: 1>:176: 2022-09-01 16:02:58 ###场景四: 双向挂单 限价 买/卖 ( 不成交)###
2022-09-01 16:02:58,369:INFO:restAcc:1498226913.py:4:QryBalAccOrd:176: 2022-09-01 16:02:58 查询组合: 14 , 余额+账户数据/持仓+订单
2022-09-01 16:03:09,273:INFO:restAcc:584110258.py:54:QryBalance:176: df_b :
  asset      balance  availableBalance     updateTime           queryTime
0  USDT  3934.652588       3731.880127  1661990400123 2022-09-01 10:10:48
1  USDT  3934.652588       3731.880127  1661990400123 2022-09-01 10:41:03
2  USDT  3934.652588       3731.880127  1661990400123 2022-09-01 15:09:40
3  USDT  3934.612793       3731.880127  1662019200231 2022-09-01 16:03:08

 df_balance: 
     accountAlias asset  availableBalance      balance  crossUnPnl  crossWalletBalance  marginAvailable  maxWithdrawAmount           queryTime              updateDate     updateTime
0  FzuXfWTioCmYmY  USDT       3731.880127  3934.652588         0.0         3731.880127             True        3731.88

In [ ]:
###交易条件单 order###订单###TradeFuture###

timestamp = int(round((time.time() +serverTimeadj+1) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/order'
urlTest = "http://httpbin.org/post"
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = 'btcusdt'  #交易对 
side = 'BUY'        #买卖方向 BUY 买入, SELL 卖出
orderType = 'LIMIT'      #订单类型 LIMIT限价单, MARKET 市价单, STOP止损限价单, STOP_MARKET止损市价单, TAKE_PROFIT止盈限价单, TAKE_PROFIT_MARKET止盈市价单, TRAILING_STOP_MARKET跟踪止损单,
price = 19000.1  #委托价格
quantity = 0.001  #下单数量
timeInForce = 'GTC' #有效方法 GTC-Good Till Cancel 成交为止, IOC-Immediate or Cancel 无法立即成交(吃单)的部分就撤销, FOK-Fill or Kill 无法全部立即成交就撤销, GTX-Good Till Crossing 无法成为挂单方就撤销

reduceOnly = 'false' #仅减仓 false true 
workingType = 'CONTRACT_PRICE'  #stopPrice 触发类型: MARK_PRICE(标记价格), CONTRACT_PRICE(合约最新价). 默认 CONTRACT_PRICE
priceProtect = 'false' #条件单触发保护："TRUE","FALSE", 默认"FALSE". 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
newOrderRespType = "ACK"  #"ACK", "RESULT", 默认 "ACK"

sideDict = {'BUY':'B', 'SELL':'S', }
typeDict = {'LIMIT':'LMT', 'MARKET':'MKT', 'STOP':'STP','STOP_MARKET':'STP','TAKE_PROFIT':'TKP','TAKE_PROFIT_MARKET':'TPM','TRAILING_STOP_MARKET':'TSM', }
reduceOnlyDict = {'false':0, 'true':1,}
workingTypeDict = {'CONTRACT_PRICE':'C', 'MARK_PRICE':'M',}
priceProtectDict = {'false':0, 'true':1,}
newOrderRespTypeDict = {'ACK':'A', 'RESULT':'R',}

positionSide = 'BOTH' #持仓方向，单向持仓模式下非必填，默认且仅可填BOTH;在双向持仓模式下必填,且仅可选择 LONG 或 SHORT, BOTH 单一持仓方向, LONG 多头(双向持仓下), SHORT 空头(双向持仓下)
callbackRate = 5 # 追踪止损回调比例，可取值范围[0.1, 5],其中 1代表1% ,仅TRAILING_STOP_MARKET 需要此参数
stopPrice = 20000   #触发价, 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
closePosition = 'true' #触发后全部平仓，true, false；仅支持STOP_MARKET和TAKE_PROFIT_MARKET；不与quantity合用；自带只平仓效果，不与reduceOnly 合用
activationPrice = 20000 #追踪止损激活价格，仅TRAILING_STOP_MARKET 需要此参数, 默认为下单当前市场价格(支持不同workingType)

closePositionDict = {'false':0, 'true':1,}
positionSideDict = {'BOTH':'B', 'LONG':'L', 'SHORT':'S', }

tailStr = '%d%s%d%s%d%s' % (reduceOnlyDict[reduceOnly], workingTypeDict[workingType], priceProtectDict[priceProtect], newOrderRespTypeDict[newOrderRespType], closePositionDict[closePosition], positionSideDict[positionSide] )
symbolStr = '%s-------' % (symbol.lower() )
tacticName = 'T0'
newClientOrderId = '%s%s%s%s%s%s%s' % (tacticName, sideDict[side], time.strftime("%Y%m%d%H%M%S"), typeDict[orderType], symbolStr[:7], timeInForce, tailStr )  #自定义订单号 ^[\.A-Z\:/a-z0-9_-]{1,36}$    2+1+14+3 +7+3+6

query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))
secret = secret_key
def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
# params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
params = {"newClientOrderId":newClientOrderId,"symbol":symbol,"side":side,"type":orderType,"price":price,"quantity":quantity,"timeInForce":timeInForce, "recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}

headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

# res = requests.post(url, headers=headers, params=params) #, params=params  urlTest  url2 
res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

logger.info(f'url2 : {url2}')

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else :
    del rt['cumQty'] #弃用
    rt_df = pd.DataFrame([rt])

    rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
    rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
    # rt_df['time'] = timestamp

    rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
    rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
    rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
    rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
    rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
    rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
    # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
    # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

    rt_df.sort_index(axis=1, ascending=True, inplace=True)

    if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values:
        df_order.sort_index(axis=1, ascending=True, inplace=True)
        df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
    else:
        df_order = pd.concat([df_order,rt_df ], ignore_index=True)

    orderId = rt['orderId']

    df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

    conn = sqlite3.connect(dbfile)
    # ##df_order = pd.read_sql(' select * from orders ', conn) 
    rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
    conn.close()

    df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
    # ##df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

    logger.info(f'df_o :\n{df_o}')
        
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****交易条件单 order###订单###TradeFuture***** ')        


Now: 1.0.1 : 2022-08-31 10:17:18 *****交易 order###订单###TradeFuture***** 


In [ ]:
###查询账户数据 account###余额assets+持仓positions###


timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v2/account'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

res = requests.get(url,headers=headers,params=params)
rt = json.loads(res.text)

print(f'res.text :\n{res.text}')

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else :

    for ass in rt["assets"]:

        if ass['asset'] != 'USDT':
            continue 

        rt_df = pd.DataFrame([ass])

        # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

        rt_df['availableBalance']  = pd.to_numeric(rt_df['availableBalance'] , errors='coerce', downcast='float')
        rt_df['crossUnPnl']  = pd.to_numeric(rt_df['crossUnPnl'] , errors='coerce', downcast='float')
        rt_df['crossWalletBalance']  = pd.to_numeric(rt_df['crossWalletBalance'] , errors='coerce', downcast='float')
        rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
        rt_df['marginBalance']  = pd.to_numeric(rt_df['marginBalance'] , errors='coerce', downcast='float')
        rt_df['maxWithdrawAmount']  = pd.to_numeric(rt_df['maxWithdrawAmount'] , errors='coerce', downcast='float')
        rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
        rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
        rt_df['unrealizedProfit']  = pd.to_numeric(rt_df['unrealizedProfit'] , errors='coerce', downcast='float')
        rt_df['walletBalance']  = pd.to_numeric(rt_df['walletBalance'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)
        df_account = pd.concat([df_account,rt_df ], ignore_index=True)
        df_a = df_account[['asset','walletBalance','availableBalance','maintMargin','marginBalance','unrealizedProfit','updateDate','queryTime', ] ]

        conn = sqlite3.connect(dbfile)
        # df_account = pd.read_sql(' select * from account ', conn) 
        rt_df.to_sql('account', con=conn, if_exists='append', index=False) #  replace替换 append追加
        conn.close()

        df_account.to_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
        # df_acc = pd.read_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )

        logger.info(f'df_a :\n{df_a}')

    for posi in rt["positions"]:

        if not abs(float(posi['positionAmt'])) > 0 :
            continue 

        rt_df = pd.DataFrame([posi])

        # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

        rt_df['initialMargin']  = pd.to_numeric(rt_df['initialMargin'] , errors='coerce', downcast='float')
        rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
        rt_df['unrealizedProfit']  = pd.to_numeric(rt_df['unrealizedProfit'] , errors='coerce', downcast='float')
        rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
        rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
        rt_df['entryPrice']  = pd.to_numeric(rt_df['entryPrice'] , errors='coerce', downcast='float')
        rt_df['maxNotional']  = pd.to_numeric(rt_df['maxNotional'] , errors='coerce', downcast='float')
        rt_df['bidNotional']  = pd.to_numeric(rt_df['bidNotional'] , errors='coerce', downcast='float')
        rt_df['askNotional']  = pd.to_numeric(rt_df['askNotional'] , errors='coerce', downcast='float')
        rt_df['positionAmt']  = pd.to_numeric(rt_df['positionAmt'] , errors='coerce', downcast='float')

        rt_df['leverage']  = pd.to_numeric(rt_df['leverage'] , errors='coerce', downcast='integer')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)
        df_position = pd.concat([df_position, rt_df ], ignore_index=True)
        df_p = df_position[['symbol','initialMargin','maintMargin','unrealizedProfit','positionInitialMargin','openOrderInitialMargin','leverage','entryPrice','maxNotional','positionAmt','notional','updateDate','queryTime', ] ]

        conn = sqlite3.connect(dbfile)
        # df_position = pd.read_sql(' select * from position ', conn) 
        rt_df.to_sql('position', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_position.to_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )
        # df_posi = pd.read_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )

        logger.info(f'df_p :\n{df_p}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****查询账户数据 account###余额assets+持仓positions***** ')    

In [ ]:
###倒计时撤销所有订单 countdownCancelAll###订单合并###0###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/countdownCancelAll'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
# orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
# origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

# orderId = 3204222910
# origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

countdownTime = 0 #倒计时。1000表示1秒； 0表示取消倒计时撤单功能。可反复调用 1000 * 300

query_string = 'symbol=%s&countdownTime=%d&recvWindow=%s&timestamp=%s' % (symbol,countdownTime,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"countdownTime":countdownTime,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

# res = requests.post(url, headers=headers, params=params) #, params=params  urlTest  url2 
res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

In [6]:
###调整开仓杠杆 leverage###1x
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/leverage'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

# symbol = 'btcusdt'
leverage = 1   #目标杠杆倍数：1 到 125 整数

query_string = 'symbol=%s&leverage=%d&recvWindow=%s&timestamp=%s' % (symbol,leverage,recvWindow,str(timestamp))

secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"leverage":leverage,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)  # {"symbol":"BTCUSDT","leverage":2,"maxNotionalValue":"15000000"}

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

2022-09-26 01:07:31,970:ERROR:restAcc:3897204470.py:37:<cell line: 36>:13154: res : {"code":-1102,"msg":"Mandatory parameter 'symbol' was not sent, was empty/null, or malformed."}


In [21]:
###变换逐全仓模式 marginType###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/marginType'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = 'btcusdt'
marginType = 'ISOLATED'   #保证金模式 ISOLATED(逐仓), CROSSED(全仓)

query_string = 'symbol=%s&marginType=%s&recvWindow=%s&timestamp=%s' % (symbol,marginType,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"marginType":marginType,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

2022-09-01 12:47:12,550:ERROR:restAcc:2135745808.py:36:<cell line: 35>:176: res : {"code":-4046,"msg":"No need to change margin type."}


In [19]:
###查询持仓模式 positionSide###1x
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/positionSide/dual'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"


query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-08-31 14:31:05,386:INFO:restAcc:3277230769.py:37:<cell line: 34>:355: res : {"dualSidePosition":false}


In [ ]:
###更改持仓模式 positionSide###1x###POST###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/positionSide/dual'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

dualSidePosition = "false"   #"true": 双向持仓模式；"false": 单向持仓模式

query_string = 'dualSidePosition=%s&recvWindow=%s&timestamp=%s' % (dualSidePosition,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"dualSidePosition":dualSidePosition,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

In [20]:
###查询联合保证金模式 multiAssetsMargin###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/multiAssetsMargin'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"


query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-08-31 14:33:48,020:INFO:restAcc:1771671977.py:37:<cell line: 34>:355: res : {"multiAssetsMargin":false}


In [ ]:
###更改联合保证金模式 multiAssetsMargin###POST###  
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/multiAssetsMargin'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"


multiAssetsMargin = "false"   #"true": 联合保证金模式开启；"false": 联合保证金模式关闭

query_string = 'multiAssetsMargin=%s&recvWindow=%s&timestamp=%s' % (multiAssetsMargin,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"multiAssetsMargin":multiAssetsMargin,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

In [27]:
###用户手续费率 commissionRate ###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/commissionRate'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt"

query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-08-31 15:11:45,707:INFO:restAcc:1294152053.py:37:<cell line: 34>:355: res : {"symbol":"BTCUSDT","makerCommissionRate":"0.000200","takerCommissionRate":"0.000400"}


In [3]:
###杠杆分层标准 leverageBracket###get###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/leverageBracket'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt"

query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-09-26 11:56:42,152:INFO:restAcc:2283347077.py:37:<cell line: 34>:4954: res : [{"symbol":"BTCUSDT","brackets":[{"bracket":1,"initialLeverage":25,"notionalCap":50000,"notionalFloor":0,"maintMarginRatio":0.01,"cum":0.0},{"bracket":2,"initialLeverage":25,"notionalCap":250000,"notionalFloor":50000,"maintMarginRatio":0.02,"cum":500.0},{"bracket":3,"initialLeverage":10,"notionalCap":1000000,"notionalFloor":250000,"maintMarginRatio":0.05,"cum":8000.0},{"bracket":4,"initialLeverage":5,"notionalCap":2000000,"notionalFloor":1000000,"maintMarginRatio":0.1,"cum":58000.0},{"bracket":5,"initialLeverage":4,"notionalCap":5000000,"notionalFloor":2000000,"maintMarginRatio":0.125,"cum":108000.0},{"bracket":6,"initialLeverage":3,"notionalCap":10000000,"notionalFloor":5000000,"maintMarginRatio":0.1665,"cum":315500.0},{"bracket":7,"initialLeverage":2,"notionalCap":15000000,"notionalFloor":10000000,"maintMarginRatio":0.25,"cum":1150500.0},{"bracket":8,"initialLeverage":1,"notionalCap":50000000,"notionalF

In [36]:
###持仓ADL队列估算 adlQuantile###get###每30秒更新数据###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/adlQuantile'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt" #队列分数0，1，2，3，4，分数越高说明在ADL队列中的位置越靠前

query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-08-31 15:33:15,699:INFO:restAcc:1632398734.py:37:<cell line: 34>:355: res : {}


In [41]:
###用户持仓风险V2 positionRisk###get###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/positionRisk'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt"

query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-08-31 18:05:27,444:INFO:restAcc:149600916.py:37:<cell line: 34>:355: res : [{"symbol":"BTCUSDT","positionAmt":"0.020","entryPrice":"20198.9","markPrice":"20187.63043601","unRealizedProfit":"-0.22539127","liquidationPrice":"10209.62581818","leverage":"2","maxNotionalValue":"15000000","marginType":"isolated","isolatedMargin":"201.60201753","isAutoAddMargin":"false","positionSide":"BOTH","notional":"403.75260872","isolatedWallet":"201.82740880","updateTime":1661933391670}]


In [33]:
###账户成交历史 userTrades###get###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/userTrades'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt"
# startTime = 1661843145792
# endTime = 1661844145792
# fromId = 123456  #返回该fromId及之后的成交，缺省返回最近的成交
limit = 2

query_string = 'symbol=%s&limit=%s&recvWindow=%s&timestamp=%s' % (symbol,limit,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"limit":limit,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-08-31 15:26:23,233:INFO:restAcc:1601264391.py:41:<cell line: 38>:355: res : [{"symbol":"BTCUSDT","id":240769077,"orderId":3205410420,"side":"BUY","price":"20373.60","qty":"0.002","realizedPnl":"-0.74080000","marginAsset":"USDT","quoteQty":"40.74720","commission":"0.01629888","commissionAsset":"USDT","time":1661843145792,"positionSide":"BOTH","maker":false,"buyer":true},{"symbol":"BTCUSDT","id":240769078,"orderId":3205410420,"side":"BUY","price":"20379","qty":"0.001","realizedPnl":"-0.37580000","marginAsset":"USDT","quoteQty":"20.37900","commission":"0.00815160","commissionAsset":"USDT","time":1661843145792,"positionSide":"BOTH","maker":false,"buyer":true}]


In [16]:
###获取账户损益资金流水 income###get###最近3个月###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/income'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt"
# startTime = 1661843145792
# endTime = 1661844145792
# incomeType = "REALIZED_PNL"  #收益类型 "TRANSFER"划转，"WELCOME_BONUS"体验金，"REALIZED_PNL"已实现盈亏，"FUNDING_FEE"资金费用，"COMMISSION"手续费，"INSURANCE_CLEAR"爆仓清算，"REFERRAL_KICKBACK"推荐人返佣，"COMMISSION_REBATE"被推荐人返现，"DELIVERED_SETTELMENT"下架结算，"COIN_SWAP_DEPOSIT"资产转换转入，"COIN_SWAP_WITHDRAW" 资产转换转出，"AUTO_EXCHANGE"
limit = 2

query_string = 'symbol=%s&limit=%s&recvWindow=%s&timestamp=%s' % (symbol,limit,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"limit":limit,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

2022-09-02 08:18:20,950:INFO:restAcc:2844515195.py:41:<cell line: 38>:204: res : [{"symbol":"BTCUSDT","incomeType":"FUNDING_FEE","income":"-0.03955160","asset":"USDT","time":1662048000000,"info":"FUNDING_FEE","tranId":6177505153939134200,"tradeId":""},{"symbol":"BTCUSDT","incomeType":"FUNDING_FEE","income":"0.02848943","asset":"USDT","time":1662076800000,"info":"FUNDING_FEE","tranId":6178713113575020720,"tradeId":""}]


In [ ]:
###查询当前挂单 openOrder###订单合并###QryOpenOrder###
def QryOpenOrder():

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/openOrder'
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
    # origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    # orderId = 3204222910
    # origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    query_string = 'symbol=%s&orderId=%s&recvWindow=%s&timestamp=%s' % (symbol,orderId,recvWindow,str(timestamp))
    secret = secret_key

    def hashing(query_string):
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

    signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
    # print(f'{signature}')
    params = {"symbol":symbol,"orderId":orderId,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')

        rt_df.sort_index(axis=1, ascending=True, inplace=True)
        
        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values: 
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from orders ', conn) 
        rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****查询当前挂单 openOrder###订单合并###QryOpenOrder***** ')  

In [ ]:
###查查询所有订单+历史 allOrders###订单合并###err404###
import hashlib
import hmac
import time
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/allOrders '
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
# orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
# origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

# orderId = 3204222910
# origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

startTime = 0  # 起始时间, 查询时间范围最大不得超过7天, 默认查询最近7天内的数据
endTime = 0 # 结束时间
limit = 10 # 返回的结果集数量 默认值:500 最大值:1000

query_string = 'symbol=%slimit=%d&recvWindow=%s&timestamp=%s' % (symbol,limit,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

res = requests.get(url,headers=headers,params=params)
rt = json.loads(res.text)

print(f'{rt = }')

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else :
    for order in rt:
        rt_df = pd.DataFrame([order])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms") 
        # rt_df['queryTime'] = time.strftime("%Y-%m-%d %H:%M:%S")
        rt_df['time'] = timestamp

        rt_df.sort_index(axis=1, ascending=True, inplace=True)
        
        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values: 
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from orders ', conn) 
        rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

    logger.info(f'df_order :\n{df_order}')


In [ ]:
###查询行情 kline###continuousKlines###
def continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',startTime=0, endTime=0, limit=499 ):

    url = f'{serverUrl}/fapi/v1/continuousKlines'

    # pair = 'btcusdt'
    # contractType = 'PERPETUAL'
    # interval = '5m' #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    # startTime = 0
    # endTime = 0
    # limit = 3 #499

    if limit < 0 or startTime < 0 or endTime < 0 or (startTime > endTime):
        k_df = pd.DataFrame()
        logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" or "startTime" or "endTime" is not valid."}}')
        return k_df

    elif limit == 0:
        if startTime == 0 and endTime == 0:
            k_df = pd.DataFrame()
            logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" is not valid."}}') 
            return k_df

        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime }  
    elif limit > 0:
        if startTime == 0 and endTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "limit":limit, }  
        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "limit":limit, }   
        elif startTime > 0 and endTime > 0:  # startTime + limit < endTime
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "endTime":endTime, "limit":limit, }   

    res = requests.get(url, params=params)

    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        k_df = pd.DataFrame()
        logger.error(f'res : {res.text}')
    else:
        k_df = pd.DataFrame(rt)
        k_df.columns = ['openTime','open','high','low','close','vol','closeTime','Amt','numTrade','bidVol','bidAmt'] #,'ignore'

        pd.to_numeric([k_df['open'], k_df['high'], k_df['low'], k_df['close'], k_df['vol'], k_df['Amt'], ], errors='coerce', downcast='float')
        
        pd.to_numeric([k_df['openTime'], k_df['closeTime'], k_df['numTrade'] ], errors='coerce', downcast='integer')

    return k_df

#权重 [1,100)  1 ; [100, 500)	2 ; [500, 1000]	5 ;  > 1000	10 ;

# [
# 1661744100000,      // 开盘时间 openTime
# "18879.99",         // 开盘价 open
# "18900.00",         // 最高价 high
# "18878.98",         // 最低价 low
# "18896.13",         // 收盘价(当前K线未结束的即为最新价) close
# "492.363",          // 成交量 vol
# 1661744399999,      // 收盘时间 closeTime
# "9302145.66080",    // 成交额 Amt
# 1874,               // 成交笔数 numTrade
# "385.983",          // 主动买入成交量 bidVol
# "7292402.33267",    // 主动买入成交额 bidAmt
# "0"                 // 请忽略该参数 ignore
# ]

# k_df = continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(limit=2, endTime =1661824559999 ) # 1661824559999 ,2 
# k_df = continuousKlines(limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(startTime =1661824259000, endTime =1661824559999 ) # 5
# k_df = continuousKlines(limit=2, startTime =1661824259000, ) # 2
# k_df = continuousKlines(limit=0, startTime =0, endTime =0 ) #err 0 
# k_df = continuousKlines(limit=-2 ) # err 0
# k_df = continuousKlines(limit=2 ) # 2
# print(f'{k_df}')  #11s

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****日志 logger***** ')

In [ ]:
k_df = continuousKlines(limit=2 ) # 2
print(f'{k_df.dtypes}')  #11s
print(f'{k_df}')  #11s

In [ ]:
###查询历史行情 kline###histUKline###
def histUKline(symbol="BTCUSDT", interval="1m", histDays=1, limit=998):

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]
    curSec = int(time.time())
    histSec = int(time.mktime(time.strptime(time.strftime("%Y%m%d", time.localtime(time.time()-60*60*24*histDays ) ), '%Y%m%d')))

    interval_pre = interval
    if interval_pre == '10m':
        interval = '5m'
        intervalSec = intervalSec//2

    totalCount = math.ceil( (curSec - histSec )/intervalSec )
    logger.info(f'{interval=}, {histSec=}, {curSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])

    count = 1
    for Sec in range(histSec, curSec, intervalSec*limit):

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000+999=}', ', endTime=', f'{(Sec+intervalSec*limit-1)*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000+999, endTime=(Sec+intervalSec*limit-1)*1000+999 )

        kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,kdf],ignore_index=True)
        del kdf

        count += 1
        time.sleep(3)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')

    if interval_pre == '10m':
        interval = interval_pre
        intervalSec = int(intervalSec*2)
        rePeriod = '10T'

        ukdfHistCp = ukdfHist.copy()
        ukdfHistCp['dateTime'] = ukdfHistCp['closeSec'].apply(lambda x: datetime.fromtimestamp(x))
        ukdfHistCp = ukdfHistCp.set_index(keys=['dateTime'], drop=True)
        # ukdfHistCp = ukdfHistCp.reindex(ukdfHistCp['dateTime'].sort_values(ascending=True).index)

        logger.info(f'ukdfHistCp.iloc[:5,:] :\n{ukdfHistCp.iloc[:5,:]}')

        if not ukdfHistCp.empty:
            openSr = ukdfHistCp['open'].resample(rePeriod, label='right').first()  
            openTimeSr = ukdfHistCp['openTime'].resample(rePeriod, label='right').first()  
            closeSr = ukdfHistCp['close'].resample(rePeriod, label='right').last()  #last first max min
            closeTimeSr = ukdfHistCp['closeTime'].resample(rePeriod, label='right').last()  
            closeSecSr = ukdfHistCp['closeSec'].resample(rePeriod, label='right').last()  
            tradeDateSr = ukdfHistCp['tradeDate'].resample(rePeriod, label='right').last()  

            ukdf = pd.concat([tradeDateSr, openTimeSr, closeTimeSr, closeSecSr,openSr, closeSr], axis=1) 
            
        ukdf.columns = ['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close']
        ukdf.reset_index(drop=True,inplace=True)

        if ukdf.iloc[-1]['closeSec'] - ukdf.iloc[-2]['closeSec'] < intervalSec:
            ukdf.drop([len(ukdf)-1],inplace=True)

        # ukdf = ukdfHistCp.copy()
        # logger.info(f'{ukdf.iloc[:5,:]=}')
        
        del ukdfHistCp

    else:
        ukdf = ukdfHist.copy()

    del ukdfHist

    ukdf['close'] = ukdf['close'].astype(float)
    ukdf['pct'] = ukdf['close']/ukdf['close'].shift(1)-1  #涨跌幅
    ukdf.fillna(0, inplace=True)

    logger.info(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
    logger.info(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

    return ukdf

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****日志 logger***** ')

In [ ]:
symbol = "BTCUSDT"
interval = "10m" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
histDays = 60  # 30 10 84 120
limit = 998 # 1500  499 998

uk_df = histUKline( interval="10m", histDays=7)
print(uk_df.shape)
print(uk_df)

In [22]:
###生成listenKey###post###无signature###  
import time
import requests
import json
from datetime import datetime 

url = f'{serverUrl}/fapi/v1/listenKey'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

headers = {"X-MBX-APIKEY":api_key}
res = requests.post(url, headers=headers) #
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

2022-08-31 14:47:41,818:INFO:restAcc:2856291003.py:18:<cell line: 15>:355: res : {"listenKey":"zwmEtcMvqzf6hL4eQKH4uNDFOfSSPF6x90p20puokPuhVNU68u70pWS3Kv85MWhp"}


In [27]:
###延长listenKey有效期###put###无signature###+60min--24h###  
import time
import requests
import json
from datetime import datetime 

url = f'{serverUrl}/fapi/v1/listenKey'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

headers = {"X-MBX-APIKEY":api_key}
res = requests.put(url, headers=headers) #
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

2022-09-02 13:51:51,407:ERROR:restAcc:3020939094.py:16:<cell line: 15>:204: res : {"code":-1125,"msg":"This listenKey does not exist."}


In [25]:
###关闭listenKey###delete###无signature###  
import time
import requests
import json
from datetime import datetime 

url = f'{serverUrl}/fapi/v1/listenKey'
api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

headers = {"X-MBX-APIKEY":api_key}
res = requests.delete(url, headers=headers) #
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

2022-08-31 14:50:26,858:INFO:restAcc:112571714.py:18:<cell line: 15>:355: res : {}


In [17]:
###最新标记价格和资金费率 premiumIndex###get###
import time
import requests
import json
from datetime import datetime 

url = f'{serverUrl}/fapi/v1/premiumIndex'

params = {"symbol":'IOSTUSDT',}  #IOSTUSDT  btcusdt

res = requests.get(url, params=params) #
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt :
    logger.error(f'res : {res.text}')
else :
    logger.info(f'res : {res.text}')

# "symbol": "BTCUSDT",                // 交易对
# "markPrice": "11793.63104562",      // 标记价格
# "indexPrice": "11781.80495970",     // 指数价格
# "estimatedSettlePrice": "11781.16138815",  // 预估结算价,仅在交割开始前最后一小时有意义
# "lastFundingRate": "0.00038246",    // 最近更新的资金费率
# "nextFundingTime": 1597392000000,   // 下次资金费时间
# "interestRate": "0.00010000",       // 标的资产基础利率
# "time": 1597370495002               // 更新时间    

2022-09-02 08:27:09,200:INFO:restAcc:2111325473.py:17:<cell line: 14>:204: res : {"symbol":"IOSTUSDT","markPrice":"0.01301328","indexPrice":"0.01301205","estimatedSettlePrice":"0.01304039","lastFundingRate":"0.00010000","interestRate":"0.00010000","nextFundingTime":1662105600000,"time":1662078428000}


In [ ]:
###查询行情 kline###continuousKlines###
def continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',startTime=0, endTime=0, limit=499 ):

    url = f'{serverUrl}/fapi/v1/continuousKlines'

    # pair = 'btcusdt'
    # contractType = 'PERPETUAL'
    # interval = '5m' #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    # startTime = 0
    # endTime = 0
    # limit = 3 #499

    if limit < 0 or startTime < 0 or endTime < 0 or (startTime > endTime):
        k_df = pd.DataFrame()
        logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" or "startTime" or "endTime" is not valid."}}')
        return k_df

    elif limit == 0:
        if startTime == 0 and endTime == 0:
            k_df = pd.DataFrame()
            logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" is not valid."}}') 
            return k_df

        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime }  
    elif limit > 0:
        if startTime == 0 and endTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "limit":limit, }  
        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "limit":limit, }   
        elif startTime > 0 and endTime > 0:  # startTime + limit < endTime
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "endTime":endTime, "limit":limit, }   

    res = requests.get(url, params=params)

    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        k_df = pd.DataFrame()
        logger.error(f'res : {res.text}')
    else:
        k_df = pd.DataFrame(rt)
        k_df.columns = ['openTime','open','high','low','close','vol','closeTime','Amt','numTrade','bidVol','bidAmt','ignore']

        pd.to_numeric([k_df['open'], k_df['high'], k_df['low'], k_df['close'], k_df['vol'], k_df['Amt'], ], errors='coerce', downcast='float')
        
        pd.to_numeric([k_df['openTime'], k_df['closeTime'], k_df['numTrade'] ], errors='coerce', downcast='integer')

    return k_df

#权重 [1,100)  1 ; [100, 500)	2 ; [500, 1000]	5 ;  > 1000	10 ;

# [
# 1661744100000,      // 开盘时间 openTime
# "18879.99",         // 开盘价 open
# "18900.00",         // 最高价 high
# "18878.98",         // 最低价 low
# "18896.13",         // 收盘价(当前K线未结束的即为最新价) close
# "492.363",          // 成交量 vol
# 1661744399999,      // 收盘时间 closeTime
# "9302145.66080",    // 成交额 Amt
# 1874,               // 成交笔数 numTrade
# "385.983",          // 主动买入成交量 bidVol
# "7292402.33267",    // 主动买入成交额 bidAmt
# "0"                 // 请忽略该参数 ignore
# ]

# k_df = continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(limit=2, endTime =1661824559999 ) # 1661824559999 ,2 
# k_df = continuousKlines(limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(startTime =1661824259000, endTime =1661824559999 ) # 5
# k_df = continuousKlines(limit=2, startTime =1661824259000, ) # 2
# k_df = continuousKlines(limit=0, startTime =0, endTime =0 ) #err 0 
# k_df = continuousKlines(limit=-2 ) # err 0
# k_df = continuousKlines(limit=2 ) # 2
# print(f'{k_df}')  #11s

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****日志 logger***** ')